In [1]:
import os
import pandas as pd
from PIL import Image

In [2]:
ROOT = os.getcwd()
combined_path_direction = 'combined-images-direction'
not_thumbnail_dir = 'combined-images-direction-not-a-thumbnail'
directions = ['front', 'back', 'side', 'mix']
processed_folders = os.listdir(os.path.join(ROOT, combined_path_direction))
processed_folders = list(filter(lambda f: f != '.DS_Store', processed_folders))

In [3]:
def not_a_thumbnail(image_path):
    im = Image.open(image_path)
    width, height = im.size
    return width > 150

In [132]:
data = []
for processed_folder in processed_folders:
    image_counts = []
    image_total = 0
    image_counts.append(processed_folder)
    for direction in directions:
        direction_path = os.path.join(ROOT, combined_path_direction, processed_folder, direction)

        image_paths = list(map(lambda s: os.path.join(direction_path, s), os.listdir(direction_path)))
        with open(os.path.join(not_thumbnail_dir, f'{direction}---{processed_folder}.txt'),'w') as f:
          f.write('\n'.join(image_paths))

        image_count = len(list(filter(not_a_thumbnail, image_paths)))
        
        image_total += image_count
        image_counts.append(image_count)
    image_counts.append(image_total)
    data.append(image_counts)

In [133]:
def convert_counts(row):
    for num in range(5):
        num += 1
        row[num] = int(row[num])
    return row

def to_percentage(row):
    for num in range(4):
        num += 1
        row[num] = round(100 * row[num]/row[5], 2)
    return row

data = list(map(convert_counts, data))
data = list(map(to_percentage, data))

In [134]:
headers = ['name', 'front', 'back', 'side', 'mix', 'total']
df = pd.DataFrame(data, columns=headers)
df = df[df['total'] >= 3000]
print(f'total classes: {len(df.index)}')

front_min = df["front"].min()
front_max = df["front"].max()

back_min = df["back"].min()
back_max = df["back"].max()

side_min = df["side"].min()
side_max = df["side"].max()

mix_min = df["mix"].min()
mix_max = df["mix"].max()

df['front_count'] = round(front_min * df['total'] / 100)
df['back_count'] = round(back_min * df['total'] / 100)
df['side_count'] = round(side_min * df['total'] / 100)
df['mix_count'] = round(mix_min * df['total'] / 100)
df['min_total'] = df['front_count'] + df['back_count'] + df['side_count'] + df['mix_count']
      
# df_pre = pd.DataFrame([
#     ['Minimum', df["front"].min(), df["back"].min(), df["side"].min(), df["mix"].min(), df["total"].min(),],
#     ['Maximum', df["front"].max(), df["back"].max(), df["side"].max(), df["mix"].max(), df["total"].max(),],
# ], columns=headers)
# df = df_pre.append(df, ignore_index=True)
      
df.to_csv('image_counts.csv', sep=',', index=False)


total classes: 10


In [136]:
df

,name,front,back,side,mix,total,front_count,back_count,side_count,mix_count,min_total
26,기아_카니발_올 뉴 카니발(14~18년),28.69,27.00,3.16,41.15,3137,727.0,731.0,99.0,1158.0,2715.0
43,벤츠_E-클래스_ E-클래스 W212 (09~17년),28.31,29.83,4.10,37.76,3101,719.0,723.0,98.0,1145.0,2685.0
62,아우디_A6_뉴 A6 (04년~현재),28.13,26.41,4.42,41.04,3260,756.0,760.0,103.0,1204.0,2823.0
71,현대_그랜저_그랜저 HG(11~16년),29.82,23.31,4.24,42.63,11675,2706.0,2721.0,369.0,4310.0,10106.0
76,현대_싼타페_싼타페 CM(05~12년),23.18,28.30,6.12,42.41,4513,1046.0,1052.0,143.0,1666.0,3907.0
77,현대_싼타페_싼타페 DM(12~15년),26.38,26.94,4.23,42.45,3378,783.0,787.0,107.0,1247.0,2924.0
79,현대_쏘나타_LF 쏘나타(14~17년),31.53,28.17,3.39,36.92,4342,1006.0,1012.0,137.0,1603.0,3758.0
84,현대_에쿠스_에쿠스(신형)(09~15년),29.29,24.55,4.45,41.72,3442,798.0,802.0,109.0,1271.0,2980.0
86,현대_제네시스_제네시스 DH(13~16년),30.69,26.58,3.43,39.30,3789,878.0,883.0,120.0,1399.0,3280.0
88,현대_제네시스_제네시스(08~13년),29.93,28.07,4.63,37.38,3221,747.0,751.0,102.0,1189.0,2789.0


In [135]:
f=open('directional_image_labels.txt','w')
s1 = '\n'.join(df[
    (df['side_count'] >= 59) &
    (df['front_count'] >= 380) &
    (df['back_count'] >= 380) &
    (df['mix_count'] >= 380)
]['name'].tolist())
f.write(s1)
f.close()